In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.preprocess import *
from utils.model import *
import datetime
from tqdm import tqdm

In [2]:
train_data = read_data('train')
test_data = read_data('test')

In [3]:
cat_chronom_train = pd.read_csv('data_chronom/chronom_duration_train.csv')
cat_chronom_test = pd.read_csv('data_chronom/chronom_duration_test.csv')

cat_chronom_train = cat_chronom_train.set_index('NPLV')
cat_chronom_test = cat_chronom_test.set_index('NPLV')

train_gas = pd.read_csv('data/gas_train.csv')
test_gas = pd.read_csv('data/gas_test.csv')

train_gas['Time'] = pd.to_datetime(train_gas['Time'])
test_gas['Time'] = pd.to_datetime(test_gas['Time'])

train_data['chronom']['VR_NACH'] = pd.to_datetime(train_data['chronom']['VR_NACH'])
test_data['chronom']['VR_NACH'] = pd.to_datetime(test_data['chronom']['VR_NACH'])
train_data['chronom']['VR_KON'] = pd.to_datetime(train_data['chronom']['VR_KON'])
test_data['chronom']['VR_KON'] = pd.to_datetime(test_data['chronom']['VR_KON'])

In [4]:
def make_state_gas(event='Продувка', i = 1):
    row = train_data['chronom'][train_data['chronom']['NOP'] == event].set_index('NPLV')
    start_train_nach = row['VR_NACH']
    end_train_nach = row['VR_KON']

    row = test_data['chronom'][test_data['chronom']['NOP'] == event].set_index('NPLV')
    start_test_nach = row['VR_NACH']
    end_test_nach = row['VR_KON']
    
    col_gas_feat = ['V', 'T', 'O2', 'N2', 'H2', 'CO2', 'CO', 'AR', 'O2_pressure']

    type_oper = '_start_' + event
    for col in col_gas_feat:
        cat_chronom_train[col + type_oper] = 0.0
        cat_chronom_test[col + type_oper] = 0.0

    type_oper = '_end_' + event
    for col in col_gas_feat:
        cat_chronom_train[col + type_oper] = 0.0
        cat_chronom_test[col + type_oper] = 0.0
        
        
    for nplv_obj in tqdm(train_gas['NPLV'].unique()):
        try:
            df_nplv = train_gas[(train_gas['NPLV'] == nplv_obj) & \
                                ((train_gas['Time'] - start_train_nach[nplv_obj]).dt.total_seconds() <= 1) & \
                                ((train_gas['Time'] - start_train_nach[nplv_obj]).dt.total_seconds() >= 0)]
            type_oper = '_start_' + event
            for gas_feat in col_gas_feat:
                cat_chronom_train.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat]
        except:
            pass
        
    for nplv_obj in tqdm(test_gas['NPLV'].unique()):
        try:
            df_nplv = test_gas[(test_gas['NPLV'] == nplv_obj) & \
                                ((test_gas['Time'] - start_test_nach[nplv_obj]).dt.total_seconds() <= 1) & \
                                ((test_gas['Time'] - start_test_nach[nplv_obj]).dt.total_seconds() >= 0)]
            type_oper = '_start_' + event
            for gas_feat in col_gas_feat:
                cat_chronom_test.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat] 
        except:
            pass
        
    for nplv_obj in tqdm(train_gas['NPLV'].unique()):
        try:
            df_nplv = train_gas[(train_gas['NPLV'] == nplv_obj) & \
                                ((end_train_nach[nplv_obj] - train_gas['Time']).dt.total_seconds() <= 1) & \
                                ((end_train_nach[nplv_obj] - train_gas['Time']).dt.total_seconds() >= 0)]
            type_oper = '_end_' + event
            for gas_feat in col_gas_feat:
                cat_chronom_train.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat]
        except:
            pass 
    for nplv_obj in tqdm(test_gas['NPLV'].unique()):
        try:
            df_nplv = test_gas[(test_gas['NPLV'] == nplv_obj) & \
                                ((end_test_nach[nplv_obj] - test_gas['Time']).dt.total_seconds() <= 1) & \
                                ((end_test_nach[nplv_obj] - test_gas['Time']).dt.total_seconds() >= 0)]
            type_oper = '_end_' + event
            for gas_feat in col_gas_feat:
                cat_chronom_test.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat] 
        except:
            pass
        
    cat_chronom_train.to_csv('data_chronom/chronom_duration_train_new' + str(i) + '.csv')
    cat_chronom_test.to_csv('data_chronom/chronom_duration_test_new' + str(i) + '.csv')

In [7]:
make_state_gas('Продувка', 1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 780/780 [00:46<00:00, 16.93it/s]


In [8]:
cat_chronom_train = pd.read_csv('data_chronom/chronom_duration_train_new1.csv')
cat_chronom_test = pd.read_csv('data_chronom/chronom_duration_test_new1.csv')

cat_chronom_train = cat_chronom_train.set_index('NPLV')
cat_chronom_test = cat_chronom_test.set_index('NPLV')

In [9]:
make_state_gas('Заливка чугуна', 2)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 780/780 [00:46<00:00, 16.82it/s]


In [10]:
cat_chronom_train = pd.read_csv('data_chronom/chronom_duration_train_new2.csv')
cat_chronom_test = pd.read_csv('data_chronom/chronom_duration_test_new2.csv')

cat_chronom_train = cat_chronom_train.set_index('NPLV')
cat_chronom_test = cat_chronom_test.set_index('NPLV')

In [11]:
make_state_gas('Завалка лома', 3)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 780/780 [00:46<00:00, 16.81it/s]
